In [2]:
# importing basic necessities like pandas, numpy, matplotlib and seaborn
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('books_cleaned.csv')
df.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."


In [4]:
df['categories'].value_counts().reset_index().query('count>=30')

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [5]:
category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Children's Fiction",
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Children's Nonfiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction",
 'Literary Collections': "Fiction",
 'Business & Economics': "Nonfiction",
 'Social Science': "Nonfiction",
 'Performing Arts': "Nonfiction",
 'Cooking': "Nonfiction",	
 'Psychology' : "Nonfiction",
 'Art': "Nonfiction",
 'Body, Mind & Spirit' : "Nonfiction",
 'Travel' : "Nonfiction",
 'Political Science' : "Nonfiction"
  }

df["simple_categories"] = df["categories"].map(category_mapping)

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

main_categories = ['Fiction',"Nonfiction"]
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

c:\Users\admin\Documents\BookRecom\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


In [7]:
# var stores first value of the description column
var = df.loc[df['simple_categories'] == 'Fiction', 'description'].reset_index(drop=True)[0]
pipe(var, main_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [8]:
# max_index = np.argmax(pipe(var, main_categories)['scores']) # get the index of the highest score
# max_label = pipe(var,main_categories)['labels'][max_index] # get the label of the highest score
# max_label

In [9]:
# create a function and bundle above to get the category of the book
def func_predict_category(var, main_categories):
    predictions = pipe(var, main_categories)
    max_index = np.argmax(predictions['scores']) # get the index of the highest score
    max_label = predictions['labels'][max_index] # get the label of the highest score
    return max_label

In [10]:
# from tqdm import tqdm   

# actual_categories = []
# predicted_categories = []
# # Fiction
# for i in tqdm(range(0,200)):
#     sequence = df.loc[df['simple_categories'] == 'Fiction', 'description'].reset_index(drop=True)[i]
#     predicted_categories += [func_predict_category(sequence, main_categories)]
#     actual_categories += ['Fiction']

# # Nonfiction
# for i in tqdm(range(0,200)):
#     sequence = df.loc[df['simple_categories'] == 'Nonfiction', 'description'].reset_index(drop=True)[i]
#     predicted_categories += [func_predict_category(sequence, main_categories)]
#     actual_categories += ['Nonfiction'] 

In [11]:
# predicted_df = pd.DataFrame({'actual_categories': actual_categories, 'predicted_categories': predicted_categories})
# predicted_df
# predicted_df['correctness'] = predicted_df['actual_categories'] == predicted_df['predicted_categories']
# # check the correctness of the predictions
# predicted_df['correctness'].value_counts()
# # percentage of correctness
# predicted_df['correctness'].value_counts(normalize=True) * 100

In [12]:
isbns = []
predicted_cats = []

missing_cats = df.loc[df["simple_categories"].isna(), ["isbn13", "description"]].reset_index(drop=True)

In [13]:
len(missing_cats)

1073

In [14]:
# below we are using the function to predict the category of the missing categories
from tqdm import tqdm   
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["description"][i]
    predicted_cats += [func_predict_category(sequence, main_categories)]
    isbns += [missing_cats["isbn13"][i]]

100%|██████████| 1073/1073 [1:16:55<00:00,  4.30s/it] 


In [15]:
missing_predicted_df = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_cats})
missing_predicted_df

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1068,9788125026600,Nonfiction
1069,9788171565641,Fiction
1070,9788172235222,Fiction
1071,9788173031014,Nonfiction


In [16]:
# here we are merging the missing predicted categories with the original dataframe using the isbn13 column as common column
df = pd.merge(df, missing_predicted_df, on="isbn13", how="left")
df["simple_categories"] = np.where(df["simple_categories"].isna(), df["predicted_categories"], df["simple_categories"])
df = df.drop(columns = ["predicted_categories"])

In [23]:
missing_predicted_df

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1068,9788125026600,Nonfiction
1069,9788171565641,Fiction
1070,9788172235222,Fiction
1071,9788173031014,Nonfiction


In [21]:
missing_predicted_df[missing_predicted_df["predicted_categories"].str.lower().isin([
    "romance",
    "science fiction",
    "scifi",
    "fantasy",
    "horror",
    "mystery",
    "thriller",
    "comedy",
    "crime",
    "historical"
])]

,isbn13,predicted_categories


In [22]:
# save the dataframe to a csv file
df.to_csv("books_with_categories.csv", index=False)